# 1.0 - Human as a tool

Occasionally, there will be questions that the Language Model cannot address, and it will request human input. It could be something personal, related to a friend or family member, or any other topic requiring human insight. In these instances, you can employ 'Human as a tool'.

In [7]:
from langchain import OpenAI, SerpAPIWrapper, LLMMathChain, WikipediaAPIWrapper
from langchain.agents import load_tools, initialize_agent, AgentType, tools
from langchain.chat_models import ChatOpenAI
from langchain_experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner

from dotenv import load_dotenv
import warnings
import pprint

warnings.filterwarnings('ignore')
load_dotenv()

True

In [2]:
llm = OpenAI(temperature = 0)
tools = load_tools(["human"])
agent_chain = initialize_agent(
    tools,
    llm,
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose = True
)

agent_chain.run("What's my friend Andi's surname")



> Entering new AgentExecutor chain...
 I should ask a human for guidance
Action: human
Action Input: What's my friend Andi's surname?

What's my friend Andi's surname?


 Neves



Observation: Neves
Thought: I now know the final answer
Final Answer: Neves

> Finished chain.


'Neves'

# 2.0 Accessing Updated Information via Wikipedia

It's well-known that most available LLMs were trained with data up until September 2021. However, there are instances where accurate and up-to-date responses are required, especially for information that emerged after this date. A practical solution is to direct the LLM to consult current sources like Wikipedia, Google Search, Bing Search, among others, to retrieve the most recent information.

In the example beloI we demonstrate how to pose a question and search for the updated answer wikipedia.dia

In [3]:
prompt = "When was the 3rd president of the united states born? What is that raised to the power of 3?"

llm_search = OpenAI(temperature = 0)
tools_search = load_tools(['wikipedia','llm-math'], llm=llm_search)
agent = initialize_agent(tools_search, llm_search, agent="zero-shot-react-description", verbose=True)

In [4]:
agent.run(prompt)



> Entering new AgentExecutor chain...
 I should use Wikipedia to find the birthdate of the 3rd president and then use a calculator to raise it to the power of 3.
Action: Wikipedia
Action Input: "3rd president of the united states"
Observation: Page: Thomas Jefferson
Summary: Thomas Jefferson (April 13, 1743 – July 4, 1826) was an American statesman, diplomat, lawyer, architect, philosopher, and Founding Father who served as the third president of the United States from 1801 to 1809. He was the primary author of the Declaration of Independence. Following the American Revolutionary War and prior to becoming president in 1801, Jefferson was the nation's first U.S. secretary of state under George Washington and then the nation's second vice president under John Adams.
During the American Revolution, Jefferson represented Virginia at the Second Continental Congress and served as the second governor of Virginia from 1779 to 1781. In 1785, Congress appointed Jefferson U.S. minister to Franc

'Thomas Jefferson was born on April 13, 1743 and when raised to the power of 3, it is 5,832,087,107.'

# 3.0 Accessing Updated Information via Google Search

To answer the question bellow, the LLM needs to:
 - Search fot the location
 - Identify the country
 - Search for the population of the country
 - Raise the population to 0.43
 - Return answer


In [10]:
llm_google_search = OpenAI(temperature = 0)
llm_math_chain = LLMMathChain.from_llm(llm = llm_google_search, verbose=True)
google_search = SerpAPIWrapper()
wikipedia = WikipediaAPIWrapper()

tools_search = [
    tools.Tool(
        name="Search",
        func = google_search.run,
        description = "useful for when you need to answer questions about current events"
    ),
    tools.Tool(
        name="Search",
        func = wikipedia.run,
        description = "useful for when you need to look up facts and statistics"
    ),
    tools.Tool(
        name="Calculator",
        func = llm_math_chain.run,
        description = "useful for when you need to answer questions about math"
    )
]

prompt = "Where are the next supper olympics going to be hosted? What is the population of that country raised to the 0.43 power?"

model = ChatOpenAI(temperature=0)
planner = load_chat_planner(model)
executor = load_agent_executor(model, tools_search, verbose=True)

In [11]:
agent = PlanAndExecute(planner=planner, executor=executor, verbose=True)
agent.run(prompt)



> Entering new PlanAndExecute chain...
steps=[Step(value='Research the location of the next summer Olympics.'), Step(value='Find the population of that country.'), Step(value='Calculate the population raised to the power of 0.43.'), Step(value="Given the above steps taken, respond to the user's original question.\n\n")]

> Entering new AgentExecutor chain...
Thought: To find the location of the next summer Olympics, I can use the Search tool.

Action:
```{"action": "Search", "action_input": "location of next summer Olympics"}```
Observation: Page: 2024 Summer Olympics
Summary: The 2024 Summer Olympics (French: Jeux olympiques d'été de 2024), officially the Games of the XXXIII Olympiad (French: Jeux de la XXXIIIe Olympiade) and commonly known as Paris 2024, is an upcoming international multi-sport event scheduled to take place from 26 July to 11 August 2024 in France, with Paris as the main host city and 16 other cities spread across metropolitan France, plus one subsite in Tahiti—an 

'Based on the research, the location of the next summer Olympics is Paris, France. The 2024 Summer Olympics, also known as Paris 2024, is scheduled to take place from 26 July to 11 August 2024 in Paris, with events also being held in 16 other cities across metropolitan France and one subsite in Tahiti. Paris 2024 will mark the centenary of the 1924 Summer Olympics held in Paris. The population of France is approximately 68,035,000 inhabitants. The population of France raised to the power of 0.43 is approximately 2333.8606804501287.'